# Assignment 4
## Econ 8310 - Business Forecasting

This assignment will make use of the bayesian statistical models covered in Lessons 10 to 12.

A/B Testing is a critical concept in data science, and for many companies one of the most relevant applications of data-driven decision-making. In order to improve product offerings, marketing campaigns, user interfaces, and many other user-facing interactions, scientists and engineers create experiments to determine the efficacy of proposed changes. Users are then randomly assigned to either the treatment or control group, and their behavior is recorded.
If the changes that the treatment group is exposed to can be measured to have a benefit in the metric of interest, then those changes are scaled up and rolled out to across all interactions.
Below is a short video detailing the A/B Testing process, in case you want to learn a bit more:
[https://youtu.be/DUNk4GPZ9bw](https://youtu.be/DUNk4GPZ9bw)

For this assignment, you will use an A/B test data set, which was pulled from the Kaggle website (https://www.kaggle.com/datasets/yufengsui/mobile-games-ab-testing). I have added the data from the page into Codio for you. It can be found in the cookie_cats.csv file in the file tree. It can also be found at [https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/cookie_cats.csv](https://github.com/dustywhite7/Econ8310/raw/master/AssignmentData/cookie_cats.csv)

The variables are defined as follows:

| Variable Name  | Definition |
|----------------|----|
| userid         | A unique number that identifies each player  |
| version        | Whether the player was put in the control group (gate_30 - a gate at level 30) or the group with the moved gate (gate_40 - a gate at level 40) |
| sum_gamerounds | The number of game rounds played by the player during the first 14 days after install.  |
| retention1     | Did the player come back and play 1 day after installing?     |
| retention7     | Did the player come back and play 7 days after installing?    |               

### The questions

You will be asked to answer the following questions in a small quiz on Canvas:
1. What was the effect of moving the gate from level 30 to level 40 on 1-day retention rates?
2. What was the effect of moving the gate from level 30 to level 40 on 7-day retention rates?
3. What was the biggest challenge for you in completing this assignment?

You will also be asked to submit a URL to your forked GitHub repository containing your code used to answer these questions.

In [3]:
import numpy as np
import pandas as pd

# 1. Load data
url = "https://raw.githubusercontent.com/dustywhite7/Econ8310/master/AssignmentData/cookie_cats.csv"
df = pd.read_csv(url)

# Fix column names
rename_dict = {}
if "retention_1" in df.columns and "retention1" not in df.columns:
    rename_dict["retention_1"] = "retention1"
if "retention_7" in df.columns and "retention7" not in df.columns:
    rename_dict["retention_7"] = "retention7"
if rename_dict:
    df = df.rename(columns=rename_dict)

print("Columns:", list(df.columns))


# 2. Bayesian A/B test function
def ab_test_mc(df,
               version_col,
               outcome_col,
               control_label,
               treatment_label,
               alpha_prior=1.0,
               beta_prior=1.0,
               n_draws=10_000,
               random_seed=8310):
    rng = np.random.default_rng(random_seed)

    control = df[df[version_col] == control_label][outcome_col]
    treatment = df[df[version_col] == treatment_label][outcome_col]

    control_success = control.sum()
    control_total = control.count()
    treatment_success = treatment.sum()
    treatment_total = treatment.count()

    control_fail = control_total - control_success
    treatment_fail = treatment_total - treatment_success

    control_samples = rng.beta(alpha_prior + control_success,
                               beta_prior + control_fail,
                               size=n_draws)
    treatment_samples = rng.beta(alpha_prior + treatment_success,
                                 beta_prior + treatment_fail,
                                 size=n_draws)

    prob_treatment_better = np.mean(treatment_samples > control_samples)

    control_mean = control_samples.mean()
    treatment_mean = treatment_samples.mean()
    diff_samples = treatment_samples - control_samples
    diff_mean = diff_samples.mean()
    diff_ci_low, diff_ci_high = np.percentile(diff_samples, [2.5, 97.5])

    result = {
        "control_mean": control_mean,
        "treatment_mean": treatment_mean,
        "diff_mean": diff_mean,
        "diff_ci_low": diff_ci_low,
        "diff_ci_high": diff_ci_high,
        "prob_treatment_better": prob_treatment_better,
        "control_total": int(control_total),
        "treatment_total": int(treatment_total),
    }
    return result


# 3. 1-day retention(retention1)
res1 = ab_test_mc(
    df=df,
    version_col="version",
    outcome_col="retention1",
    control_label="gate_30",
    treatment_label="gate_40",
)

print("=== 1-day retention (retention1) ===")
print(f"Control (gate_30) posterior mean:   {res1['control_mean']:.4f}")
print(f"Treatment (gate_40) posterior mean: {res1['treatment_mean']:.4f}")
print(f"Difference (gate_40 - gate_30):     {res1['diff_mean']:.4f}")
print(f"95% CI for difference: [{res1['diff_ci_low']:.4f}, {res1['diff_ci_high']:.4f}]")
print(f"P(gate_40 > gate_30):               {res1['prob_treatment_better']:.3f}")
print(f"Sample sizes - control: {res1['control_total']}, treatment: {res1['treatment_total']}")
print()


# 4. 7-day retention (retention7)
res7 = ab_test_mc(
    df=df,
    version_col="version",
    outcome_col="retention7",
    control_label="gate_30",
    treatment_label="gate_40",
)

print("=== 7-day retention (retention7) ===")
print(f"Control (gate_30) posterior mean:   {res7['control_mean']:.4f}")
print(f"Treatment (gate_40) posterior mean: {res7['treatment_mean']:.4f}")
print(f"Difference (gate_40 - gate_30):     {res7['diff_mean']:.4f}")
print(f"95% CI for difference: [{res7['diff_ci_low']:.4f}, {res7['diff_ci_high']:.4f}]")
print(f"P(gate_40 > gate_30):               {res7['prob_treatment_better']:.3f}")
print(f"Sample sizes - control: {res7['control_total']}, treatment: {res7['treatment_total']}")


Columns: ['userid', 'version', 'sum_gamerounds', 'retention1', 'retention7']
=== 1-day retention (retention1) ===
Control (gate_30) posterior mean:   0.4482
Treatment (gate_40) posterior mean: 0.4423
Difference (gate_40 - gate_30):     -0.0059
95% CI for difference: [-0.0125, 0.0005]
P(gate_40 > gate_30):               0.036
Sample sizes - control: 44700, treatment: 45489

=== 7-day retention (retention7) ===
Control (gate_30) posterior mean:   0.1902
Treatment (gate_40) posterior mean: 0.1820
Difference (gate_40 - gate_30):     -0.0082
95% CI for difference: [-0.0133, -0.0032]
P(gate_40 > gate_30):               0.001
Sample sizes - control: 44700, treatment: 45489
